Load Features

In [16]:
import pandas as pd
import matplotlib as plt

In [12]:
data = pd.read_csv('output.csv',header=0,sep=';')

In [15]:
data['frameTime']

0      0.000000
1      0.010023
2      0.020045
3      0.030068
4      0.040091
5      0.050113
6      0.060136
7      0.070159
8      0.080181
9      0.090204
10     0.100227
11     0.110249
12     0.120272
13     0.130295
14     0.140317
15     0.150340
16     0.160363
17     0.170385
18     0.180408
19     0.190431
20     0.200454
21     0.210476
22     0.220499
23     0.230522
24     0.240544
25     0.250567
26     0.260590
27     0.270612
28     0.280635
29     0.290658
         ...   
913    9.150703
914    9.160726
915    9.170748
916    9.180771
917    9.190794
918    9.200816
919    9.210839
920    9.220862
921    9.230884
922    9.240907
923    9.250930
924    9.260952
925    9.270975
926    9.280998
927    9.291020
928    9.301043
929    9.311066
930    9.321088
931    9.331111
932    9.341134
933    9.351156
934    9.361179
935    9.371202
936    9.381224
937    9.391247
938    9.401270
939    9.411293
940    9.421315
941    9.431338
942    9.441361
Name: frameTime, Length:

In [17]:
x = data.size

In [18]:
x

2829

In [ ]:
data.